# Dataset manipulation (nibabel)


## longueur du jeu et config chemins

In [7]:

import nibabel as nib
import numpy as np
import os


%matplotlib inline
import matplotlib.pyplot as plt

import os
print("Répertoire de travail actuel :", os.getcwd())
print(os.listdir("../..")) #premier test


Répertoire de travail actuel : /home/tibia/Documents/nnUNet/nnUNet_V_act/ich_seg_v4_adrien-20250318T153056Z-001/ich_seg_v4_adrien


In [8]:
#travail sur les donnees de test
root_folder= '/home/tibia/Documents/nnUNet/nnUNet_V_act/splitv2/ich_seg_v4_adrien'

for subfolder in ['imagesTr','labelsTr']:
    folder_path=os.path.join(root_folder,subfolder)
    files=os.listdir(folder_path)
    print(f'nombre de fichiers dans {folder_path}= {len(files)}')
    print('-------------------------------')


nombre de fichiers dans /home/tibia/Documents/nnUNet/nnUNet_V_act/ich_seg_v4_adrien-20250318T153056Z-001/ich_seg_v4_adrien/imagesTr= 341
-------------------------------
nombre de fichiers dans /home/tibia/Documents/nnUNet/nnUNet_V_act/ich_seg_v4_adrien-20250318T153056Z-001/ich_seg_v4_adrien/labelsTr= 341
-------------------------------


In [2]:
import os
#changer jjuste le root ou tu veux mettre au bon format
root_folder = '/home/tibia/Documents/nnUNet/splitv2/ich_seg_v4_adrien/'
images_folder = os.path.join(root_folder, 'imagesTs')

# Vérifie que le dossier existe
if os.path.exists(images_folder):
    files = os.listdir(images_folder)
    print(f"il y a {len(files)} fichiers dans {images_folder}")
    for file in files:
        if file.endswith('.nii.gz'):  # Vérifie le format d'image attendu
            # Sépare le nom et l'extension
            base, ext = os.path.splitext(file)

            # Si l'extension est .gz, traite correctement le double suffixe
            if base.endswith('.nii'):
                base, nii_ext = os.path.splitext(base)
                ext = nii_ext + ext

            # Vérifie si le suffixe _0000 est déjà présent
            if not base.endswith('_0000'):
                new_name = f"{base}_0000{ext}"
                old_path = os.path.join(images_folder, file)
                new_path = os.path.join(images_folder, new_name)

                # Renomme le fichier
                os.rename(old_path, new_path)
                print(f"Renommé : {file} → {new_name}")
            else:
                print(f"Déjà au format nnU-Net : {file}")
else:
    print(f"Le dossier {images_folder} n'existe pas.")


il y a 18 fichiers dans /home/tibia/Documents/nnUNet/splitv2/ich_seg_v4_adrien/imagesTs
Renommé : ID_7f88806c.nii.gz → ID_7f88806c_0000.nii.gz
Renommé : ID_02599242.nii.gz → ID_02599242_0000.nii.gz
Renommé : ID_3909b107.nii.gz → ID_3909b107_0000.nii.gz
Renommé : ID_3b4accbe.nii.gz → ID_3b4accbe_0000.nii.gz
Renommé : ID_28d52dd2.nii.gz → ID_28d52dd2_0000.nii.gz
Renommé : ID_4d990362.nii.gz → ID_4d990362_0000.nii.gz
Renommé : ID_3604e294.nii.gz → ID_3604e294_0000.nii.gz
Renommé : ID_76aeaf2d.nii.gz → ID_76aeaf2d_0000.nii.gz
Renommé : ID_ad9019df.nii.gz → ID_ad9019df_0000.nii.gz
Renommé : ID_730cb33c.nii.gz → ID_730cb33c_0000.nii.gz
Renommé : ID_0634c835.nii.gz → ID_0634c835_0000.nii.gz
Renommé : ID_8e954b69.nii.gz → ID_8e954b69_0000.nii.gz
Renommé : ID_a6c3a453.nii.gz → ID_a6c3a453_0000.nii.gz
Renommé : ID_22257cef.nii.gz → ID_22257cef_0000.nii.gz
Renommé : ID_747f1c99.nii.gz → ID_747f1c99_0000.nii.gz
Renommé : ID_21daa8ce.nii.gz → ID_21daa8ce_0000.nii.gz
Renommé : ID_40d6277f.nii.gz → I

In [ ]:


# Lister les fichiers dans chaque dossier
train_files = set(os.listdir(root_train))  # Fichiers dans imagesTr
seg_files = set(os.listdir(root_seg))      # Fichiers dans labelsTr

# Afficher le nombre de fichiers dans chaque dossier
print(f"Nombre de fichiers dans imagesTr : {len(train_files)}")
print(f"Nombre de fichiers dans labelsTr : {len(seg_files)}")
print('-------------------------------')

# Vérifier les fichiers manquants
missing_in_seg = train_files - seg_files  # Fichiers dans imagesTr mais pas dans labelsTr
missing_in_train = seg_files - train_files  # Fichiers dans labelsTr mais pas dans imagesTr

# Afficher les fichiers manquants
if missing_in_seg:
    print("Fichiers manquants dans labelsTr :")
    for file in sorted(missing_in_seg):
        print(file)
else:
    print("Aucun fichier manquant dans labelsTr.")

print('-------------------------------')

if missing_in_train:
    print("Fichiers manquants dans imagesTr :")
    for file in sorted(missing_in_train):
        print(file)
else:
    print("Aucun fichier manquant dans imagesTr.")


In [ ]:
#Premiere manip
file = "ID_7f88806c.nii.gz" 
file_path= os.path.join(os.path.join(root_folder,'img'),file)
print("Fichier trouvé :", os.path.exists(file_path))

img=nib.load(file_path)
header=img.header
print(header)
print(header.get_data_shape())
print(header.get_data_dtype())
print(header.get_zooms())

In [ ]:
img.dataobj
print(img.get_fdata())

## statistic generation

In [11]:
def load_nifti(file_path):
    img = nib.load(file_path)
    return img.get_fdata(), img  

# Calculer les statistiques demandées
def compute_statistics(image_array):
    min_val = np.min(image_array)
    max_val = np.max(image_array)
    mean_val = np.mean(image_array)

    non_zero_pixels = image_array[image_array > 0]
    non_zero_mean = np.mean(non_zero_pixels) if non_zero_pixels.size > 0 else 0

    p_05 = np.percentile(image_array, 0.5)
    p_995 = np.percentile(image_array, 99.5)

    return {
        "min": min_val,
        "max": max_val,
        "mean": mean_val,
        "non_zero_mean": non_zero_mean,
        "0.5% percentile": p_05,
        "99.5% percentile": p_995
    }

# Exemple d'utilisation avec un fichier .nii.gz

image_array, nib_image = load_nifti(file_path)
stats = compute_statistics(image_array)

# Afficher les résultats
print(stats)

{'min': -1024.0, 'max': 1668.0, 'mean': -633.3997630557499, 'non_zero_mean': 166.25389056828413, '0.5% percentile': -1012.0, '99.5% percentile': 919.0}


In [12]:
#meme chose pour l'image segmentee
file = "ID_b65b5eef.nii.gz" 
file_path_2= os.path.join(os.path.join(root_folder,'seg'),file)
print("Fichier trouvé :", os.path.exists(file_path_2))

segmentation_array, nib_image_2 = load_nifti(file_path_2)
stats = compute_statistics(segmentation_array)
print(stats)

Fichier trouvé : True
{'min': 0.0, 'max': 3.0, 'mean': 0.005533456802368164, 'non_zero_mean': 2.518064446132147, '0.5% percentile': 0.0, '99.5% percentile': 0.0}


In [13]:
unique_values, counts = np.unique(segmentation_array, return_counts=True)
print("Valeurs uniques :", unique_values)
print("Nombre de voxels par valeur :", counts)

Valeurs uniques : [0. 2. 3.]
Nombre de voxels par valeur : [8370174    8884    9550]


## Windowing


In [33]:
def apply_hu_windowing(image_array, hu_low, hu_high):
    return np.clip(image_array, hu_low, hu_high)

 
hu_low, hu_high = 30, 40 #pour cerveau : matiere blanche-matiere grise
windowed_image = apply_hu_windowing(image_array, hu_low, hu_high)


segmentation_windowed = np.clip(segmentation_array, 1, 3) #on prend que ce qui est segmentee


output_dir = os.path.join(os.path.expanduser("~"), "Downloads")
output_path_image = os.path.join(output_dir, "image_windowed.nii.gz")
output_path_segmentation = os.path.join(output_dir, "segmentation_windowed.nii.gz")

def save_nifti(image_array, reference_nib, output_path):
    new_img = nib.Nifti1Image(image_array, affine=reference_nib.affine, header=reference_nib.header)
    nib.save(new_img, output_path)
    print(f"✅ Image sauvegardée sous : {output_path}")


save_nifti(windowed_image, nib_image, output_path_image)
save_nifti(segmentation_windowed, nib_image_2, output_path_segmentation)

✅ Image sauvegardée sous : /home/tibia/Downloads/image_windowed.nii.gz
✅ Image sauvegardée sous : /home/tibia/Downloads/segmentation_windowed.nii.gz
